In [28]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [6]:
# Path ke direktori dataset
dataset_dir = 'D:/@KULIAH SMTR 6/BANGKIT/CAPSTONE/dataset/dataset/'

In [59]:
# Membaca dataset dan melakukan ekstraksi fitur menggunakan FLBP
def extract_features(dataset_dir):
    images = []
    labels = []

    for filename in os.listdir(dataset_dir):
        if filename.endswith('.bmp'):
            image_path = os.path.join(dataset_dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (100, 100))  # Mengubah ukuran gambar menjadi 100x100 pixel
            
            # Ekstraksi fitur menggunakan FLBP
            radius = 3
            n_points = 8 * radius
            lbp = local_binary_pattern(image, n_points, radius, method='uniform')
            feature = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))[0]
            
            images.append(feature)
            
            # Mendapatkan angka kolestrol dari nama file
            cholesterol = filename.split('-')[1]
            if cholesterol.isdigit():
                labels.append(int(cholesterol))
            else:
                print("Invalid filename format:", filename)

    return np.array(images), np.array(labels)

In [20]:
# Memanggil fungsi untuk melakukan ekstraksi fitur dari dataset menggunakan FLBP
images, labels = extract_features(dataset_dir)

In [22]:
images

array([[ 409,  271,  218,  161,  154,  114,  145,  142,  171,  266,  325,
         443,  605,  526,  259,  190,   96,  119,   97,  102,  103,  106,
         150,  241,  428, 4159],
       [ 390,  213,  201,  185,  153,  161,  157,  228,  247,  262,  361,
         514,  638,  567,  305,  241,  172,  156,  139,  133,  126,  114,
         135,  172,  392, 3638],
       [ 464,  300,  192,  154,  105,  112,  111,  138,  190,  184,  245,
         338,  404,  375,  186,  144,   97,   88,   86,   84,   84,  116,
         155,  275,  492, 4881],
       [ 426,  259,  163,  155,  154,  156,  139,  142,  143,  185,  251,
         379,  581,  451,  187,  171,  128,  102,  100,  100,  100,  103,
         171,  244,  438, 4572],
       [ 417,  278,  236,  160,  138,  129,  124,  125,  164,  167,  257,
         435,  657,  529,  226,  177,  123,   96,   95,  115,  103,  123,
         158,  231,  451, 4286],
       [ 400,  297,  220,  195,  141,  144,  166,  169,  188,  192,  241,
         277,  428,  

In [23]:
# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [24]:
# Membangun model regresi linear
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [34]:
# Lakukan prediksi pada data uji
y_pred = model.predict(X_test)



# Konversi prediksi menjadi kelas menggunakan threshold tertentu (jika diperlukan)
# Misalnya, jika tingkat kolestrol >= 200 dianggap tinggi, maka:
threshold = 200
y_pred_classes = (y_pred >= threshold).astype(int)
y_test_classes = (y_test >= threshold).astype(int)

# Evaluasi model
confusion_mat = confusion_matrix(y_test_classes, y_pred_classes)
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes)
recall = recall_score(y_test_classes, y_pred_classes)
f1 = f1_score(y_test_classes, y_pred_classes)


In [60]:
print("Confusion Matrix:")
print(confusion_mat)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


Confusion Matrix:
[[2 1]
 [3 2]]
Accuracy: 0.5
Precision: 0.6666666666666666
Recall: 0.4
F1-Score: 0.5


In [31]:
# Evaluasi model
score = model.score(X_test, y_test)
print("Akurasi model:", score)

# Try to predict

In [49]:
# Path gambar yang ingin diperiksa
image_path = r'D:\@KULIAH SMTR 6\BANGKIT\CAPSTONE\dataset\dataset\normal-180-20-kanan.bmp'

# Membaca gambar menggunakan cv2.imread()
image = cv2.imread(image_path)

# Memeriksa apakah gambar berhasil dibaca
if image is not None:
    print("Gambar berhasil dibaca.")
    cv2.imshow("Gambar", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Gagal membaca gambar.")

Gambar berhasil dibaca.


In [40]:
from skimage.feature import local_binary_pattern

In [51]:
# Coba Predict data
# Memuat gambar
image_path = r'D:\@KULIAH SMTR 6\BANGKIT\CAPSTONE\dataset\dataset\normal-180-20-kanan.bmp'
image = cv2.imread(image_path)

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (100, 100))  # Mengubah ukuran gambar menjadi 100x100 pixel
            
# Ekstraksi fitur menggunakan FLBP
radius = 3
n_points = 8 * radius
lbp = local_binary_pattern(image, n_points, radius, method='uniform')
feature = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))[0]

# Mengubah dimensi fitur menjadi bentuk yang dapat diterima oleh model (sesuaikan dengan bentuk fitur yang digunakan saat melatih model)
feature = np.reshape(feature, (1, -1))

# Melakukan prediksi menggunakan model
prediction = model.predict(feature)

# Cetak hasil prediksi
print("Prediksi Kolesterol:", prediction)

Prediksi Kolesterol: [15.69335938]


In [58]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [62]:
# Save Model menggunakan joblib
import joblib

# Simpan model ke dalam file .h5 menggunakan joblib
joblib.dump(model, 'regression_model.h5')

['regression_model.h5']

In [64]:
# Muat kembali model dari file .h5
model = joblib.load('regression_model.h5')

In [3]:
def extract_features(image):
    # Implementasikan ekstraksi fitur sesuai kebutuhan Anda
    # Misalnya, menggunakan FLBP seperti yang telah dibahas sebelumnya
    # atau metode ekstraksi fitur lainnya seperti HOG, SIFT, dll.
    # Di sini, kita menggunakan FLBP sebagai contoh
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Lakukan ekstraksi fitur FLBP
    flbp_features = fuzzy_local_binary_pattern(gray_image)

    return flbp_features

In [ ]:
import cv2
import numpy as np
import pandas as pd

def extract_features(image):
    # Implementasikan ekstraksi fitur sesuai kebutuhan Anda
    # Misalnya, menggunakan FLBP seperti yang telah dibahas sebelumnya
    # atau metode ekstraksi fitur lainnya seperti HOG, SIFT, dll.
    # Di sini, kita menggunakan FLBP sebagai contoh
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Lakukan ekstraksi fitur FLBP
    flbp_features = fuzzy_local_binary_pattern(gray_image)

    return flbp_features

# Path ke dataset gambar mata dan kolestrolnya (CSV)
dataset_path = 'dataset.csv'

# Membaca dataset menggunakan pandas
dataset = pd.read_csv(dataset_path)

# Inisialisasi array untuk menyimpan fitur dan label
features = []
labels = []

# Loop melalui setiap data gambar mata dalam dataset
for index, row in dataset.iterrows():
    image_path = row['image_path']
    cholesterol = row['cholesterol']

    # Baca gambar mata dari path
    image = cv2.imread(image_path)

    # Ekstraksi fitur dari gambar mata
    image_features = extract_features(image)

    # Menambahkan fitur dan label ke dalam array
    features.append(image_features)
    labels.append(cholesterol)

# Mengubah array fitur dan label menjadi bentuk yang dapat digunakan untuk pelatihan model
features = np.array(features)
labels = np.array(labels)
